In [1]:
from collections import OrderedDict
import os
import glob
import numpy as np
import pandas as pd
import dask
from dask.delayed import delayed
from dask.distributed import Client, wait
import dask_cudf
import datetime

In [2]:
print(datetime.datetime.utcnow().isoformat())

2019-11-25T20:46:07.502032


In [3]:
!cat start_dask.yaml

device_memory_limit_gib: 26.0
# Use below for Parquet
#docker_image: claudiofahey/rapidsai:a359097c3c18a534b91557d5abe772c73ef57d11de3dfb632e1516b0a01745f1
# Use below for ORC
docker_image: claudiofahey/rapidsai:0.10-cuda10.0-runtime-ubuntu18.04-custom
host:
  - 10.200.11.12
  - 10.200.11.13
  - 10.200.11.11
memory_limit_gib: 64.0


In [4]:
use_local_cuda_cluster = False
# Use below for a local-only CUDA cluster
if use_local_cuda_cluster:
    from dask_cuda import LocalCUDACluster
    cluster = LocalCUDACluster(ip='0.0.0.0')
    client = Client(cluster)

In [5]:
# Use below for a multi-host multi-GPU CUDA cluster started with start_dask.py.
if not use_local_cuda_cluster:
    scheduler_address = '10.200.11.12:8786'
    client = Client(address=scheduler_address)

In [6]:
client

Client Scheduler: tcp://10.200.11.12:8786 Dashboard: http://10.200.11.12:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [7]:
import cudf

In [9]:
%%time
# Restart all workers. This also clears GPU memory.
client.restart()

CPU times: user 9.02 ms, sys: 6.88 ms, total: 15.9 ms
Wall time: 17.8 s


Client Scheduler: tcp://10.200.11.12:8786 Dashboard: http://10.200.11.12:8787/status,Cluster Workers: 48 Cores: 240 Memory: 3.30 TB


In [8]:
def gpu_load_performance_data(performance_path, **kwargs):
    """ Loads performance data

    Returns
    -------
    GPU DataFrame
    """
    
    cols = [
        "loan_id", "monthly_reporting_period", "servicer", "interest_rate", "current_actual_upb",
        "loan_age", "remaining_months_to_legal_maturity", "adj_remaining_months_to_maturity",
        "maturity_date", "msa", "current_loan_delinquency_status", "mod_flag", "zero_balance_code",
        "zero_balance_effective_date", "last_paid_installment_date", "foreclosed_after",
        "disposition_date", "foreclosure_costs", "prop_preservation_and_repair_costs",
        "asset_recovery_costs", "misc_holding_expenses", "holding_taxes", "net_sale_proceeds",
        "credit_enhancement_proceeds", "repurchase_make_whole_proceeds", "other_foreclosure_proceeds",
        "non_interest_bearing_upb", "principal_forgiveness_upb", "repurchase_make_whole_proceeds_flag",
        "foreclosure_principal_write_off_amount", "servicing_activity_indicator"
    ]

    ddf = dask_cudf.read_orc(performance_path)
    # Fix column names from ORC file
    ddf = ddf.rename(columns=dict(zip(ddf.columns, cols)))
    return ddf

In [9]:
# Identify list of files to load.
data_dir = '/mnt/isilon1/data/mortgage'
perf_file = []
perf_file += glob.glob(data_dir + '/perf-snappy.orc/*')
# perf_file = perf_file[0:2]
len(perf_file)

96

In [10]:
%%time
perf_ddf = gpu_load_performance_data(perf_file)

CPU times: user 1.44 s, sys: 1.11 s, total: 2.55 s
Wall time: 2.97 s


In [11]:
perf_ddf

,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,current_loan_delinquency_status,mod_flag,zero_balance_code,zero_balance_effective_date,last_paid_installment_date,foreclosed_after,disposition_date,foreclosure_costs,prop_preservation_and_repair_costs,asset_recovery_costs,misc_holding_expenses,holding_taxes,net_sale_proceeds,credit_enhancement_proceeds,repurchase_make_whole_proceeds,other_foreclosure_proceeds,non_interest_bearing_upb,principal_forgiveness_upb,repurchase_make_whole_proceeds_flag,foreclosure_principal_write_off_amount,servicing_activity_indicator
npartitions=96,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,datetime64[ns],int32,float64,float64,float64,float64,float64,datetime64[ns],float64,int32,int32,int32,datetime64[ns],datetime64[ns],datetime64[ns],datetime64[ns],float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,int32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
perf_ddf.head()

,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,holding_taxes,net_sale_proceeds,credit_enhancement_proceeds,repurchase_make_whole_proceeds,other_foreclosure_proceeds,non_interest_bearing_upb,principal_forgiveness_upb,repurchase_make_whole_proceeds_flag,foreclosure_principal_write_off_amount,servicing_activity_indicator
0,709011185621,2002-05-01,null,7.0,null,2.0,358.0,358.0,2032-03-01,12060.0,...,null,null,null,null,null,null,null,null,null,null
1,709011185621,2003-05-01,null,7.0,91594.85,14.0,346.0,346.0,2032-03-01,12060.0,...,null,null,null,null,null,null,null,null,null,null
2,709011185621,2004-05-01,null,7.0,90573.25,26.0,334.0,334.0,2032-03-01,12060.0,...,null,null,null,null,null,null,null,null,null,null
3,709011185621,2005-05-01,null,7.0,89477.78,38.0,322.0,322.0,2032-03-01,12060.0,...,null,null,null,null,null,null,null,null,null,null
4,709011185621,2006-05-01,null,7.0,88303.13,50.0,310.0,310.0,2032-03-01,12060.0,...,null,null,null,null,null,null,null,null,null,null


In [13]:
%%time
# Read from files into GPU memory.
perf_ddf = perf_ddf.persist()
wait(perf_ddf)

CPU times: user 624 ms, sys: 20.1 ms, total: 644 ms
Wall time: 19.1 s


In [14]:
perf_ddf

,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,current_loan_delinquency_status,mod_flag,zero_balance_code,zero_balance_effective_date,last_paid_installment_date,foreclosed_after,disposition_date,foreclosure_costs,prop_preservation_and_repair_costs,asset_recovery_costs,misc_holding_expenses,holding_taxes,net_sale_proceeds,credit_enhancement_proceeds,repurchase_make_whole_proceeds,other_foreclosure_proceeds,non_interest_bearing_upb,principal_forgiveness_upb,repurchase_make_whole_proceeds_flag,foreclosure_principal_write_off_amount,servicing_activity_indicator
npartitions=96,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,datetime64[ns],int32,float64,float64,float64,float64,float64,datetime64[ns],float64,int32,int32,int32,datetime64[ns],datetime64[ns],datetime64[ns],datetime64[ns],float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32,float64,int32
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [15]:
#perf_ddf.dask

In [16]:
%%time
perf_ddf.head()

CPU times: user 91.1 ms, sys: 2.41 ms, total: 93.5 ms
Wall time: 737 ms


,loan_id,monthly_reporting_period,servicer,interest_rate,current_actual_upb,loan_age,remaining_months_to_legal_maturity,adj_remaining_months_to_maturity,maturity_date,msa,...,holding_taxes,net_sale_proceeds,credit_enhancement_proceeds,repurchase_make_whole_proceeds,other_foreclosure_proceeds,non_interest_bearing_upb,principal_forgiveness_upb,repurchase_make_whole_proceeds_flag,foreclosure_principal_write_off_amount,servicing_activity_indicator
0,709011185621,2002-05-01,null,7.0,null,2.0,358.0,358.0,2032-03-01,12060.0,...,null,null,null,null,null,null,null,null,null,null
1,709011185621,2003-05-01,null,7.0,91594.85,14.0,346.0,346.0,2032-03-01,12060.0,...,null,null,null,null,null,null,null,null,null,null
2,709011185621,2004-05-01,null,7.0,90573.25,26.0,334.0,334.0,2032-03-01,12060.0,...,null,null,null,null,null,null,null,null,null,null
3,709011185621,2005-05-01,null,7.0,89477.78,38.0,322.0,322.0,2032-03-01,12060.0,...,null,null,null,null,null,null,null,null,null,null
4,709011185621,2006-05-01,null,7.0,88303.13,50.0,310.0,310.0,2032-03-01,12060.0,...,null,null,null,null,null,null,null,null,null,null


In [17]:
%%time
len(perf_ddf)

CPU times: user 23.7 ms, sys: 3.58 ms, total: 27.3 ms
Wall time: 219 ms


1890353680

In [18]:
%%time
perf_ddf.groupby(['servicer'])['interest_rate'].max().compute().head(2)

CPU times: user 282 ms, sys: 51.4 ms, total: 334 ms
Wall time: 1.67 s


servicer
2128592246    3.625
2119093407    6.625
Name: interest_rate, dtype: float64

In [19]:
%%time
perf_ddf.groupby(['servicer'])['interest_rate'].max().compute().head(2)

CPU times: user 175 ms, sys: 27.5 ms, total: 202 ms
Wall time: 1.29 s


servicer
2129534051    4.875
2106898581    4.750
Name: interest_rate, dtype: float64

In [20]:
%%time
perf_ddf.groupby(['servicer'])['interest_rate'].max().compute().head(2)

CPU times: user 177 ms, sys: 27.4 ms, total: 204 ms
Wall time: 1.33 s


servicer
2119333152    6.125
2124692535    3.750
Name: interest_rate, dtype: float64

In [21]:
checksum = perf_ddf['loan_id'].sum().compute()
checksum

6573355020803881490

In [22]:
# compare to checksum from mortgage_etl_4.ipynb.
checksum - 6573355020803881490

0

In [23]:
print(datetime.datetime.utcnow().isoformat())

2019-11-25T20:47:00.954924


In [24]:
!nvidia-smi

Mon Nov 25 20:47:01 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM3...  On   | 00000000:34:00.0 Off |                    0 |
| N/A   36C    P0    82W / 350W |  10005MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM3...  On   | 00000000:36:00.0 Off |                    0 |
| N/A   35C    P0    82W / 350W |   8634MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T